# SO101 + SmolVLA 训练和推理教程

本教程包括：

1. ✅ 配置训练参数
2. ✅ 微调 SmolVLA 模型
3. ✅ 监控训练进度
4. ✅ 评估模型性能
5. ✅ 部署到实体机械臂

**前置条件**:
- ✅ 已完成数据采集 (`02_so101_data_collection.ipynb`)
- ✅ 数据集已上传到 Hugging Face Hub
- ✅ GPU 可用 (推荐 A100 / RTX 3090)

---
## 📦 步骤 0: 环境检查

In [ ]:
import torch
import os
from pathlib import Path

print("🔍 检查环境...")
print("")

# 检查 CUDA
cuda_available = torch.cuda.is_available()
print(f"CUDA 可用: {cuda_available}")
if cuda_available:
    print(f"GPU 数量: {torch.cuda.device_count()}")
    print(f"GPU 名称: {torch.cuda.get_device_name(0)}")
    print(f"CUDA 版本: {torch.version.cuda}")
else:
    print("⚠️ 没有检测到 GPU，训练会非常慢！")
    print("推荐使用 Google Colab 或云 GPU")

print("")
print(f"PyTorch 版本: {torch.__version__}")
print("")

# 检查 LeRobot
try:
    import lerobot
    print(f"✅ LeRobot 版本: {lerobot.__version__}")
except ImportError:
    print("❌ LeRobot 未安装")

# 检查 SmolVLA 依赖
try:
    from lerobot.policies.smolvla import SmolVLAPolicy
    print("✅ SmolVLA 可用")
except ImportError:
    print("❌ SmolVLA 不可用，请运行: pip install -e '.[smolvla]'")

---
## ⚙️ 步骤 1: 配置训练参数

In [ ]:
# ========== 基础配置 ==========

# Hugging Face 用户名和数据集
HF_USER = "your_hf_username"  # 修改为您的用户名
DATASET_NAME = "my_so101_pickplace"  # 您的数据集名称
DATASET_REPO_ID = f"{HF_USER}/{DATASET_NAME}"

# 任务描述（必须与数据采集时相同！）
TASK_DESCRIPTION = "Pick up the cube and place it on the plate."

# 训练配置
BATCH_SIZE = 64  # 根据您的 GPU 显存调整 (A100: 64, RTX 3090: 32-48)
TRAINING_STEPS = 20000  # 训练步数 (约 4 小时 @ A100)
LEARNING_RATE = 1e-4  # 学习率

# 输出目录
OUTPUT_DIR = Path("../models/checkpoints") / DATASET_NAME
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 模型名称
MODEL_NAME = f"smolvla_{DATASET_NAME}"

# Weights & Biases (可选)
USE_WANDB = True  # 是否使用 W&B 监控训练
WANDB_PROJECT = "lerobot-so101"
WANDB_RUN_NAME = f"smolvla_{DATASET_NAME}"

print("✅ 训练配置：")
print(f"  - 数据集: {DATASET_REPO_ID}")
print(f"  - Batch Size: {BATCH_SIZE}")
print(f"  - Steps: {TRAINING_STEPS}")
print(f"  - 输出目录: {OUTPUT_DIR}")
print(f"  - W&B: {USE_WANDB}")

---
## 🔐 步骤 2: 登录 Hugging Face 和 W&B

In [ ]:
# 登录 Hugging Face
from huggingface_hub import notebook_login

print("🔐 登录 Hugging Face...")
notebook_login()
print("✅ HF 登录成功！")

# 登录 Weights & Biases (可选)
if USE_WANDB:
    import wandb
    print("")
    print("🔐 登录 Weights & Biases...")
    wandb.login()
    print("✅ W&B 登录成功！")

---
## 🚀 步骤 3: 开始训练

### 3.1 方法 1: 使用命令行训练 (推荐)

**优点**:
- 更稳定
- 可以在后台运行
- 便于调试

In [ ]:
# 生成训练命令
train_command = f"""
lerobot-train \\
  --policy.path=lerobot/smolvla_base \\
  --dataset.repo_id={DATASET_REPO_ID} \\
  --batch_size={BATCH_SIZE} \\
  --steps={TRAINING_STEPS} \\
  --output_dir={OUTPUT_DIR} \\
  --job_name={MODEL_NAME} \\
  --policy.device=cuda \\
  --wandb.enable={str(USE_WANDB).lower()}
"""

if USE_WANDB:
    train_command += f" \\
  --wandb.project={WANDB_PROJECT} \\
  --wandb.run_name={WANDB_RUN_NAME}"

print("📋 训练命令：")
print("="*70)
print(train_command)
print("="*70)
print("")
print("💡 建议在终端运行此命令，以便：")
print("  - 实时查看训练日志")
print("  - 使用 tmux/screen 在后台运行")
print("  - 避免 Jupyter Notebook 超时")
print("")
print("📊 预计训练时间：")
print(f"  - A100 GPU: ~4 小时 ({TRAINING_STEPS} steps)")
print(f"  - RTX 3090: ~6-8 小时")
print(f"  - CPU: 不推荐 (太慢)")

### 3.2 方法 2: 在 Notebook 中训练 (实验性)

**警告**: 长时间训练可能导致 Notebook 超时或内存问题。

In [ ]:
# 如果您坚持在 Notebook 中训练（不推荐）
import subprocess

print("⚠️ 在 Notebook 中启动训练...")
print("提示：使用 Ctrl+C 可以中断训练")
print("")

# 注意：这会阻塞当前 cell 直到训练完成
# 建议使用终端运行

# subprocess.run(train_command.split(), check=True)

---
## 📊 步骤 4: 监控训练进度

### 4.1 查看 W&B Dashboard

In [ ]:
if USE_WANDB:
    print("📊 Weights & Biases Dashboard:")
    print("")
    print(f"🔗 https://wandb.ai/{HF_USER}/{WANDB_PROJECT}/runs/{WANDB_RUN_NAME}")
    print("")
    print("在 Dashboard 中可以查看：")
    print("  - Loss 曲线")
    print("  - Learning rate")
    print("  - GPU 使用率")
    print("  - 训练速度 (steps/sec)")
else:
    print("⚠️ W&B 未启用，训练日志仅在终端输出")

### 4.2 查看本地日志

In [ ]:
# 查看最新的训练日志
log_files = list(OUTPUT_DIR.glob("*.log"))

if log_files:
    latest_log = sorted(log_files, key=lambda x: x.stat().st_mtime)[-1]
    print(f"📄 最新日志文件: {latest_log}")
    print("")
    print("最后 50 行：")
    print("="*70)
    with open(latest_log, 'r') as f:
        lines = f.readlines()
        for line in lines[-50:]:
            print(line.rstrip())
else:
    print("⚠️ 没有找到日志文件")

### 4.3 查看 Checkpoint

In [ ]:
# 列出所有 checkpoint
checkpoints = sorted(OUTPUT_DIR.glob("checkpoint-*"))

print(f"📦 找到 {len(checkpoints)} 个 checkpoint:")
print("")
for cp in checkpoints:
    # 提取 step 数
    step = cp.name.split("-")[-1]
    size_mb = sum(f.stat().st_size for f in cp.rglob('*')) / (1024 * 1024)
    print(f"  - {cp.name} ({size_mb:.1f} MB)")

if checkpoints:
    print("")
    print(f"✅ 最新 checkpoint: {checkpoints[-1].name}")

---
## 🧪 步骤 5: 加载和测试训练好的模型

In [ ]:
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.factory import make_pre_post_processors

print("📦 加载训练好的模型...")
print("")

# 选择最新的 checkpoint
if checkpoints:
    model_path = str(checkpoints[-1])
    print(f"加载: {model_path}")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 加载模型
    model = SmolVLAPolicy.from_pretrained(model_path)
    model = model.to(device)
    model.eval()
    
    print("")
    print("✅ 模型加载成功！")
    print(f"  - 设备: {device}")
    print(f"  - 参数量: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")
    
    # 创建预处理器
    preprocess, postprocess = make_pre_post_processors(
        model.config,
        model_path,
        preprocessor_overrides={"device_processor": {"device": str(device)}},
    )
    
    print("✅ 预处理器创建成功！")
else:
    print("❌ 没有找到训练好的模型")
    print("请先完成训练")

---
## 🤖 步骤 6: 部署到实体机械臂

### 6.1 配置机械臂

In [ ]:
from lerobot.robots.so101_follower import SO101Follower, SO101FollowerConfig
from lerobot.cameras.opencv import OpenCVCameraConfig
from lerobot.datasets.utils import hw_to_dataset_features
from lerobot.policies.utils import build_inference_frame, make_robot_action

# 机械臂配置（从步骤 1 获取）
FOLLOWER_PORT = "/dev/ttyACM0"
FOLLOWER_ID = "my_so101_follower"

# 摄像头配置（必须与训练时相同！）
camera_configs = {
    "front": OpenCVCameraConfig(
        index_or_path=0,
        width=640,
        height=480,
        fps=30,
    ),
}

# 创建机械臂实例
robot_config = SO101FollowerConfig(
    port=FOLLOWER_PORT,
    id=FOLLOWER_ID,
    cameras=camera_configs,
)

print("✅ 机械臂配置完成")

### 6.2 连接机械臂并运行推理

In [ ]:
import time

print("🤖 开始部署...")
print("")

# 连接机械臂
robot = SO101Follower(robot_config)
robot.connect()

print("✅ 机械臂已连接")
print("")

# 特征映射
action_features = hw_to_dataset_features(robot.action_features, "action")
obs_features = hw_to_dataset_features(robot.observation_features, "observation")
dataset_features = {**action_features, **obs_features}

# 推理参数
MAX_EPISODES = 5  # 运行 5 个 episode
MAX_STEPS_PER_EPISODE = 100  # 每个 episode 最多 100 步

print(f"🎮 开始推理 ({MAX_EPISODES} episodes)...")
print("⚠️ 请确保工作空间安全！")
print("")
input("按 Enter 开始...")

try:
    for episode in range(MAX_EPISODES):
        print(f"\n📍 Episode {episode + 1}/{MAX_EPISODES}")
        
        for step in range(MAX_STEPS_PER_EPISODE):
            # 获取观测
            obs = robot.get_observation()
            
            # 构建推理帧
            obs_frame = build_inference_frame(
                observation=obs,
                ds_features=dataset_features,
                device=device,
                task=TASK_DESCRIPTION,
                robot_type="so101_follower",
            )
            
            # 预处理
            obs_processed = preprocess(obs_frame)
            
            # 模型推理
            with torch.no_grad():
                action = model.select_action(obs_processed)
            
            # 后处理
            action = postprocess(action)
            action = make_robot_action(action, dataset_features)
            
            # 发送动作
            robot.send_action(action)
            
            # 显示进度
            print(f"\rStep {step + 1}/{MAX_STEPS_PER_EPISODE}", end="", flush=True)
            
            # 控制频率
            time.sleep(0.033)  # ~30 Hz
        
        print("\n✅ Episode 完成！")
        
        if episode < MAX_EPISODES - 1:
            input("按 Enter 继续下一个 episode...")

except KeyboardInterrupt:
    print("\n⚠️ 用户中断")

finally:
    # 断开连接
    robot.disconnect()
    print("\n✅ 已安全断开连接")

---
## 📤 步骤 7: 上传模型到 Hugging Face Hub

In [ ]:
from huggingface_hub import HfApi

MODEL_REPO_ID = f"{HF_USER}/{MODEL_NAME}"

print(f"📤 上传模型到 Hub: {MODEL_REPO_ID}")
print("")

if checkpoints:
    try:
        # 上传最新的 checkpoint
        api = HfApi()
        api.upload_folder(
            folder_path=str(checkpoints[-1]),
            repo_id=MODEL_REPO_ID,
            repo_type="model",
        )
        
        print("✅ 上传成功！")
        print("")
        print(f"🔗 模型链接: https://huggingface.co/{MODEL_REPO_ID}")
    except Exception as e:
        print(f"❌ 上传失败: {e}")
else:
    print("⚠️ 没有可用的 checkpoint")

---
## ✅ 训练和部署完成！

### 总结

您已经成功完成：

1. ✅ 微调 SmolVLA 模型
2. ✅ 监控训练进度
3. ✅ 部署到实体机械臂
4. ✅ 上传模型到 Hugging Face Hub

### 性能优化建议

如果模型性能不理想：

1. **采集更多数据**：增加到 100+ episodes
2. **增加变化**：更多物体位置、姿态变化
3. **延长训练**：增加到 40k-50k steps
4. **调整超参数**：
   - 学习率：1e-5 到 5e-4
   - Batch size：根据 GPU 调整
5. **数据增强**：添加颜色、亮度变化

### 下一步

- 尝试更复杂的任务
- 多任务学习
- Sim2Real 迁移
- 与社区分享您的模型

---

**🎉 恭喜完成整个流程！**